In [1]:
import torch
from PIL import Image
import torchvision
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
IMAGES_FOLDER = './dataset/Images'
ANNOTATION_FOLDER = './dataset/Annotation'

In [3]:
images_df = pd.DataFrame()

In [4]:
def getShape(img_path):
    im = plt.imread(img_path)
    return im.shape

In [5]:
for dirpath, dirnames, filenames in os.walk(IMAGES_FOLDER):
    rows = []
    for i in filenames:
        path = os.path.join(dirpath[17:], i)
        
        w, h, _ = getShape(os.path.join(IMAGES_FOLDER, path))
        
        row = (i, dirpath[17:], path, w, h)
        
        rows.append(row)
        
    images_df = images_df.append(rows)
    
images_df.columns = ['img', 'folder', 'path', 'width', 'height']
images_df = images_df.reset_index()
images_df = images_df.drop(columns='index')

In [6]:
images_df.head()

,img,folder,path,width,height
0,n02085620_10074.jpg,n02085620-Chihuahua,n02085620-Chihuahua\n02085620_10074.jpg,500,333
1,n02085620_10131.jpg,n02085620-Chihuahua,n02085620-Chihuahua\n02085620_10131.jpg,495,395
2,n02085620_10621.jpg,n02085620-Chihuahua,n02085620-Chihuahua\n02085620_10621.jpg,298,500
3,n02085620_1073.jpg,n02085620-Chihuahua,n02085620-Chihuahua\n02085620_1073.jpg,500,345
4,n02085620_10976.jpg,n02085620-Chihuahua,n02085620-Chihuahua\n02085620_10976.jpg,484,322


In [7]:
SIZE = max(round(images_df['width'].mean()), round(images_df['height'].mean()))

In [11]:
!dir

 ’®¬ ў гбва®©бвўҐ D Ё¬ҐҐв ¬ҐвЄг Ќ®ўл© в®¬
 ‘ҐаЁ©­л© ­®¬Ґа в®¬ : EAEC-34CD

 ‘®¤Ґа¦Ё¬®Ґ Ї ЇЄЁ D:\diploma

21.04.2021  15:46    <DIR>          .
21.04.2021  15:46    <DIR>          ..
10.04.2021  09:08                28 .gitignore
20.04.2021  08:54    <DIR>          .idea
20.04.2021  10:22    <DIR>          .ipynb_checkpoints
20.04.2021  15:28    <DIR>          3semestr
20.04.2021  15:17             2я896 Annotation_changer_yolov5.ipynb
20.04.2021  15:16             2я561 class_mapper.json
20.04.2021  08:54    <DIR>          dataset
10.04.2021  09:07    <DIR>          models
21.04.2021  15:46    <DIR>          resized_dataset
21.04.2021  15:46           121я644 Untitled.ipynb
               4 д ©«®ў        127я129 Ў ©в
               8 Ї Ї®Є  246я800я916я480 Ў ©в бў®Ў®¤­®


In [50]:
#RESHAPING IMGS

NEW_IMG_FOLDER = './resized_dataset/Images'
NEW_ANN_FOLDER = './resized_dataset/Annotation'

# for img_data in images_df.values:
#     file_name = img_data[0]
#     folder_name = img_data[1]
#     path = os.path.join(IMAGES_FOLDER, img_data[2])
#     img = plt.imread(path)
#     img = cv2.resize(img, (SIZE,SIZE))
#     folder_path = os.path.join(NEW_IMG_FOLDER, folder_name)
#     file_path = os.path.join(folder_path, file_name)
#     os.makedirs(folder_path, exist_ok=True)
#     img = plt.imsave(file_path, img)
# #     break

In [52]:
#QUALITY DECREASING

# quality_list = [5, 15, 25, 35, 50, 70]

# for quality in quality_list:
    
#     qual_folder = f'./resized_dataset/Images_{quality}'
    
#     for img_data in images_df.values:
        
#         img_file_name = img_data[0]
#         img_folder_name = img_data[1]
#         img_path = img_data[2]
        
#         path = os.path.join(NEW_IMG_FOLDER, img_path)
        
#         image = Image.open(path)
        
#         folder_path = os.path.join(qual_folder, img_folder_name)
        
#         file_path = os.path.join(folder_path, img_file_name)
        
#         os.makedirs(folder_path, exist_ok = True)
        
#         image.save(file_path, "JPEG", quality = quality)